# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [ ]:
pip install requests beautifulsoup4 

In [ ]:
import requests
from bs4 import BeautifulSoup

# Set headers to mimic a real browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com'
}

url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,'

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    # Continue with your scraping logic here
else:
    print(f'Failed to retrieve the page. Status code: {response.status_code}')
soup

In [ ]:
import pandas as pd

In [ ]:
response

In [ ]:
response.content

def scrape_imdb(title_type, user_rating, start_date, end_date):
    # Prepare the URL based on the function parameters
    url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,'
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating}"
  
    # Initialize lists to store data
    movie_nr = []
    titles = []
    years = []
    ratings = []
    runtimes = []
    genres = []
    descriptions = []
    directors = []
    stars = []
    votes = []
    gross = []

    # Find all the movie items in the page
    movie_items = soup.find_all('div', class_='lister-item-content')

    # Extract data for each movie
    for idx, movie in enumerate(movie_items, start=1):
        # Movie Nr
        movie_nr.append(idx)

        # Title
        title = movie.find('a').text.strip()
        titles.append(title)

        # Year
        year = movie.find('span', class_='lister-item-year').text.strip('()')
        years.append(year)

        # IMDb Rating
        rating = movie.find('strong').text.strip()
        ratings.append(float(rating))

        # Runtime (min)
        runtime = movie.find('span', class_='runtime').text.strip()
        runtimes.append(runtime)

        # Genre
        genre = movie.find('span', class_='genre').text.strip()
        genres.append(genre)

        # Description (some movies don't have a description)
        description_tag = movie.find_all('p', class_='text-muted')
        if len(description_tag) > 1:
            description = description_tag[1].text.strip()
        else:
            description = ''
        descriptions.append(description)

        # Director and Stars
        credits = movie.find('p', class_='').text.strip().split('|')
        directors.append(credits[0].split(':')[1].strip())
        stars.append(credits[1].split(':')[1].strip())

        # Votes
        vote = movie.find_all('span', attrs={'name': 'nv'})[0].text.strip()
        votes.append(int(vote.replace(',', '')))

        # Gross earnings (some movies don't have gross)
        try:
            gross_earnings = movie.find_all('span', attrs={'name': 'nv'})[1].text.strip()
            gross.append(float(gross_earnings.strip('$M')))
        except:
            gross.append(None)

    # Create a DataFrame
    df = pd.DataFrame({
        'Movie Nr': movie_nr,
        'Title': titles,
        'Year': years,
        'Rating': ratings,
        'Runtime (min)': runtimes,
        'Genre': genres,
        'Description': descriptions,
        'Director': directors,
        'Stars': stars,
        'Votes': votes,
        'Gross ($M)': gross
    })

    return df

# Example usage:
title_type = "feature"
user_rating = 7.5
start_date = "1990-01-01"
end_date = "1992-12-31"

df = scrape_imdb(title_type, user_rating, start_date, end_date)
print(df)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,'
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
response = requests.get(url, headers=headers)

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
movie_title = soup.find_all(class_ = "ipc-title__text")
movie_title

In [ ]:
movie_title[1]

In [ ]:
movie_title[1].getText().strip()

In [ ]:
movies_names = [movie.text.strip() for movie in movie_title]
movies_names

In [ ]:
movies_names_filtered = movies_names[1:-1]
movies_names_filtered

In [ ]:
movie_nr = []
title = []

# Iterate over each line and extract the number and name
for x in movies_names_filtered:
    n= x.split(". ")[0]
    t= x.split(". ")[-1]
    movie_nr.append(n)
    title.append(t)

In [ ]:
title

In [ ]:
year_released = soup.find_all('span', class_="sc-b189961a-8 kLaxqf dli-title-metadata-item")
year_released

In [ ]:
year = []
age_rating = []
runtime = []

def extract_text_from_span(span_element):
    return span_element.get_text()

for i in range(0, len(year_released), 3):
    year_extract = year_released[i]
    runtime_extract = year_released[i+1]
    age_rating_extract = year_released[i+2]
    year.append(extract_text_from_span(year_extract))
    age_rating.append(extract_text_from_span(age_rating_extract))
    runtime.append(extract_text_from_span(runtime_extract))

In [ ]:
year

In [ ]:
age_rating

In [ ]:
runtime

In [ ]:
movies_Votes = soup.find_all('span', class_="ipc-rating-star--voteCount")
movies_Votes


In [ ]:
votes = [m_counts.text.strip().strip('()') for m_counts in movies_Votes]
votes

In [ ]:
movies_rating = soup.find_all('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating')
movies_rating

In [ ]:
rating = [span.text.strip() [:3] for span in movies_rating]
rating

In [ ]:
movies_description = soup.find_all('div', class_='ipc-html-content-inner-div')
movies_description

In [ ]:
description = [span.text.strip() for span in movies_description]
description

In [38]:
data = {
    'Movie Nr': movie_nr,
    'Title': title,
    'Year': year,
    'Rating': rating,
    'Runtime': runtime,
    'Description': description,
    'Rating': rating,
    'Votes': votes
}
def movie_data(data):
    df = pd.DataFrame(data)
    return df

movie_data(data)

,Movie Nr,Title,Year,Rating,Runtime,Description,Votes
0,1,The Silence of the Lambs,1991,8.6,1h 58m,A young F.B.I. cadet must receive the help of ...,1.6M
1,2,Goodfellas,1990,8.7,2h 25m,The story of Henry Hill and his life in the ma...,1.3M
2,3,Terminator 2: Judgment Day,1991,8.6,2h 17m,"A cyborg, identical to the one who failed to k...",1.2M
3,4,My Cousin Vinny,1992,7.6,2h,Two New Yorkers accused of murder in rural Ala...,143K
4,5,Scent of a Woman,1992,8.0,2h 36m,A prep school student needing money agrees to ...,333K
5,6,Reservoir Dogs,1992,8.3,1h 39m,When a simple jewelry heist goes horribly wron...,1.1M
6,7,Fried Green Tomatoes,1991,7.7,2h 10m,"Evelyn, an ordinary housewife, visits a nursin...",85K
7,8,The Last of the Mohicans,1992,7.6,1h 52m,Three trappers protect the daughters of a Brit...,190K
8,9,Dances with Wolves,1990,8.0,3h 1m,"Lieutenant John Dunbar, assigned to a remote w...",294K
9,10,Total Recall,1990,7.5,1h 53m,When a man goes in to have virtual vacation me...,357K
